In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["FIREWORKS_API_KEY"] = os.getenv('FIREWORKS_API_KEY')
MONGO_URI= os.getenv('MONGO_URI')

In [4]:

import pandas as pd
from datasets import load_dataset

data = load_dataset("MongoDB/subset_arxiv_papers_with_emebeddings")
dataset_df = pd.DataFrame(data["train"])

f:\GitHub\agent_with_memory\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.
Generating train split: 50000 examples [00:02, 20161.30 examples/s]


In [5]:
print(len(dataset_df))
dataset_df.head()

50000


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,embedding
0,704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...","[0.0594153292, -0.0440569334, -0.0487333685, -..."
1,704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]","[0.0247399714, -0.065658465, 0.0201423876, -0...."
2,704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]","[0.0491479263, 0.0728017688, 0.0604138002, 0.0..."
3,704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]","[0.0389556214, -0.0410280302, 0.0410280302, -0..."
4,704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]","[0.118412666, -0.0127423415, 0.1185125113, 0.0..."


In [3]:

from pymongo import MongoClient

# Initialize MongoDB python client
client = MongoClient(MONGO_URI)

DB_NAME = "agent-memory"
COLLECTION_NAME = "knowledge"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
collection = client[DB_NAME][COLLECTION_NAME]

In [4]:

# # Delete any existing records in the collection
# collection.delete_many({})

# # Data Ingestion
# records = dataset_df.to_dict('records')
# collection.insert_many(records)

# print("Data ingestion into MongoDB completed")

In [6]:

from langchain_openai import OpenAIEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=256)

# Vector Store Creation
vector_store = MongoDBAtlasVectorSearch.from_connection_string(
    connection_string=MONGO_URI,
    namespace=DB_NAME + "." + COLLECTION_NAME,
    embedding= embedding_model,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    text_key="abstract"
)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [7]:

from langchain_openai import ChatOpenAI, OpenAI
from langchain_fireworks import Fireworks, ChatFireworks

# llm = ChatFireworks(
#     model="accounts/fireworks/models/firefunction-v1",
#     max_tokens=256)

llm = ChatOpenAI()

# Agent 

In [8]:
from langchain.agents import tool, Tool
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.document_loaders import ArxivLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults

# Custom Tool Definiton
@tool
def get_metadata_information_from_arxiv(word: str) -> list:
  """
  Fetches and returns metadata for a maximum of ten documents from arXiv matching the given query word.

  Args:
    word (str): The search query to find relevant documents on arXiv.

  Returns:
    list: Metadata about the documents matching the query.
  """
  docs = ArxivLoader(query=word, load_max_docs=10).load()
  # Extract just the metadata from each document
  metadata_list = [doc.metadata for doc in docs]
  return metadata_list


@tool
def get_information_from_arxiv(word: str) -> list:
  """
  Fetches and returns metadata for a single research paper from arXiv matching the given query word, which is the ID of the paper, for example: 704.0001.

  Args:
    word (str): The search query to find the relevant paper on arXiv using the ID.

  Returns:
    list: Data about the paper matching the query.
  """
  doc = ArxivLoader(query=word, load_max_docs=1).load()
  return doc


retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="knowledge_base",
    description="This serves as the base knowledge source of the agent and contains some records of research papers from Arxiv. This tool is used as the first step for exploration and reseach efforts."
)

In [9]:
tools = [retriever_tool, get_metadata_information_from_arxiv, get_information_from_arxiv]


In [23]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_purpose = "You are a helpful research assistant"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", agent_purpose),
        # MessagesPlaceholder(
        #     variable_name="chat_history"
        # ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

# Agent Memory Using MongoDB


In [24]:

from langchain_mongodb.chat_message_histories import MongoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory

def get_session_history(session_id: str) -> MongoDBChatMessageHistory:
        return MongoDBChatMessageHistory(MONGO_URI, session_id, database_name=DB_NAME, collection_name="history")

# memory = ConversationBufferMemory(
#     memory_key="chat_history",
#     chat_memory=get_session_history("my-session")
# )

In [26]:
history_obj = get_session_history("my-session")

# Agent Creation

In [28]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    # memory=memory,
)
   

In [31]:
# human_message = "hellou mi name is damian"
# response = agent_executor.invoke({"input": human_message, "chat_history": history_obj.messages})
# ai_message = response["output"]

# history_obj.add_user_message(human_message)
# history_obj.add_ai_message(ai_message)


# response.

# history_obj.add



> Entering new AgentExecutor chain...
Hello Damian! How can I assist you today?

> Finished chain.


In [36]:
def call_agent(human_message, chat_history):

    response = agent_executor.invoke({"input": human_message, "chat_history": chat_history.messages})
    ai_message = response["output"]
    chat_history.add_user_message(human_message)
    chat_history.add_ai_message(ai_message)

    return response

In [37]:
human_message = "hellou mi name is damian"
response = call_agent(human_message, history_obj)
print(response["output"])

human_message = "do you remember my name?"
response =  call_agent(human_message, history_obj)
print(response["output"])



> Entering new AgentExecutor chain...
Hello Damian! How can I assist you today?

> Finished chain.
Hello Damian! How can I assist you today?


> Entering new AgentExecutor chain...
Yes, Damian, I remember your name. How can I help you further?

> Finished chain.
Yes, Damian, I remember your name. How can I help you further?


In [ ]:
human_message = "Get me a list of research papers on the topic Prompt Compression"
response = call_agent(human_message, history_obj)
print(response["output"])

In [ ]:
{"type": "ai", "data": {"content": "I found several research papers on the topic of Prompt Compression. Here are some of the papers with their titles and summaries:\n\n1. **PROMPT-SAW: Leveraging Relation-Aware Graphs for Textual Prompt Compression**\n   - *Authors*: Muhammad Asif Ali, Zhengping Li, Shu Yang, Keyuan Cheng, Yang Cao, Tianhao Huang, Lijie Hu, Lu Yu, Di Wang\n   - *Published*: 2024-03-30\n   - *Summary*: This paper introduces PROMPT-SAW, an effective strategy for prompt compression using relation-aware graphs. The compressed prompts generated by PROMPT-SAW outperform baseline models in terms of readability and performance.\n\n2. **Say More with Less: Understanding Prompt Learning Behaviors through Gist Compression**\n   - *Authors*: Xinze Li, Zhenghao Liu, Chenyan Xiong, Shi Yu, Yukun Yan, Shuo Wang, Ge Yu\n   - *Published*: 2024-02-25\n   - *Summary*: The Gist-COCO model is proposed for compressing prompts and assisting in prompt interpretation and engineering. It outperforms previous prompt compression models in various tasks.\n\n3. **Compress, Then Prompt: Improving Accuracy-Efficiency Trade-off of LLM Inference with Transferable Prompt**\n   - *Authors*: Zhaozhuo Xu, Zirui Liu, Beidi Chen, Yuxin Tang, Jue Wang, Kaixiong Zhou, Xia Hu, Anshumali Shrivastava\n   - *Published*: 2023-10-10\n   - *Summary*: This paper focuses on optimizing the trade-off between accuracy and efficiency by prompting compressed models. The soft prompt learning method proposed enhances the performance of compressed models.\n\nThese are just a few of the research papers on Prompt Compression. If you would like more information about any specific paper or more papers on this topic, feel free to let me know!", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": []}}

In [41]:
human_message = "Get me the abstract of the first paper you found"
response = call_agent(human_message, history_obj)
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `get_information_from_arxiv` with `{'word': '2203.15916'}`


[Document(page_content='Current Implicit Policies May Not Eradicate COVID-19\nAli Jadbabaiea,b, Arnab Sarkera, and Devavrat Shaha,c\naInstitute for Data, Systems, and Society, MIT\nbDepartment of Civil and Environmental Engineering, MIT\ncDepartment of Electrical Engineering and Computer Science, MIT\nMarch 2022\nAbstract\nSuccessful predictive modeling of epidemics requires an understanding of the implicit feedback control\nstrategies which are implemented by populations to modulate the spread of contagion. While this task\nof capturing endogenous behavior can be achieved through intricate modeling assumptions, we ﬁnd that\na population’s reaction to case counts can be described through a second order aﬃne dynamical system\nwith linear control which ﬁts well to the data across diﬀerent regions and times throughout the COVID-\n19 pandemic. The model ﬁts the data well both in 

In [ ]:
# agent_executor.invoke({"input": "do you know my name?"})

In [ ]:
# agent_executor.invoke({"input": "Get me a list of research papers on the topic Prompt Compression"})

In [ ]:
# response = agent_executor.invoke({"input":"Get me the abstract of the first paper you found"})
# response